## dependencies

In [1]:
import gym
from gym import Env 
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete


In [2]:
# import helpers
import numpy as np
import random
import os

In [3]:
# stable baselines 3 
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## Types of spaces

In [8]:
Discrete(3).sample()

0

In [16]:
Box(0,1, shape=(3,3)).sample()

array([[0.46052942, 0.76024777, 0.8744062 ],
       [0.81315106, 0.78415704, 0.37426758],
       [0.63115424, 0.07536799, 0.8385397 ]], dtype=float32)

In [18]:
Tuple((Discrete(3), Box(0,1, shape=(3,)))).sample()

(2, array([0.3259641, 0.6621263, 0.4096544], dtype=float32))

In [26]:
Dict({'height': Discrete(2), 'speed': Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([75.23421], dtype=float32))])

In [36]:
MultiBinary(4).sample()

array([1, 0, 0, 1], dtype=int8)

In [56]:
MultiDiscrete([5,2,9]).sample()

array([3, 1, 1], dtype=int64)

## building Environment
- 

In [57]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # initial shower temperature
        self.state = 38 + random.randint(-3,3)
        # Set shower length,
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0: -1 (down), 1: 0 stay, 2: +1 (up)
        self.state += action -1 
        
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            # good shower temperature
            reward =1 
        else: 
            reward = -1 
            
        if self.shower_length <=0: 
            # done showering
            done = True 
        else: 
            done = False 
            
        # Apply temperature noise
        self.state += random.randint(-1,1)
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info
    
    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        
        # Reset shower time
        self.shower_length = 60 
        
        return self.state

In [58]:
env = ShowerEnv()

c:\Users\Tinho\anaconda3\envs\gpu-env\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [59]:
env.observation_space.sample()

array([30.951935], dtype=float32)

In [75]:
env.action_space.sample()

0

## Test environment

In [93]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done: 
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

env.close()

Episode:1 Score:-42
Episode:2 Score:-38
Episode:3 Score:-24
Episode:4 Score:-26
Episode:5 Score:-16


## Train

In [94]:
log_path = os.path.join('training', 'logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\Tinho\anaconda3\envs\gpu-env\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [103]:
model.learn(total_timesteps=200000)

Logging to training\logs\PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -35      |
| time/              |          |
|    fps             | 449      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 60        |
|    ep_rew_mean          | -37.9     |
| time/                   |           |
|    fps                  | 357       |
|    iterations           | 2         |
|    time_elapsed         | 11        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0091988 |
|    clip_fraction        | 0.0118    |
|    clip_range           | 0.2       |
|    entropy_loss         | -1.09     |
|    explained_variance   | 0.000806  |
|    learning_rate        | 0

# save model

In [97]:
shower_path = os.path.join('training', 'saved_models', 'shower_model_PPO')

In [98]:
model.save(shower_path)

c:\Users\Tinho\anaconda3\envs\gpu-env\lib\site-packages\stable_baselines3\common\save_util.py:278: UserWarning: Path 'training\saved_models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [108]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(59.0, 2.4083189157584592)